<a href="https://colab.research.google.com/github/stakunlena/ich_python_for_DA_HW/blob/main/Stakun_HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Домашняя работа №5**

**1. Работа с временными рядами в pandas:**

Напишите программу, которая:
1. Создаёт DataFrame с двумя столбцами: "Дата" (в формате дд-мм-гггг) и "Значение" (целые числа).
1. Преобразует столбец "Дата" в тип datetime и устанавливает его в качестве индекса DataFrame.
1. Генерирует временной ряд, заполняя столбец "Значение" случайными числами.
1. Выводит сгенерированный временной ряд.
1. Запрашивает у пользователя начальную и конечную дату периода для фильтрации.
1. Выводит временной ряд за указанный период.

In [ ]:
import pandas as pd
import numpy as np

# 1. Создаем даты за определенный период
dates = pd.date_range(start='2023-01-01', end='2023-12-31') # берем интервал год - 365 дней

# 2. Создаем случайные значения
values = np.random.randint(100, 200, size=len(dates)) # создаем 365 случайных числе от 100 до 199

# 3. Создаем DataFrame
df = pd.DataFrame({'Дата': dates, 'Значение': values})

# 4. Преобразуем столбец "Дата" в формат datetime и устанавливаем индекс
df['Дата'] = pd.to_datetime(df['Дата'])
df.set_index('Дата', inplace=True)

# 5. Выводим сгенерированный временной ряд
print("Сгенерированный временной ряд:")
print(df)

# 6. Запрашиваем у пользователя начальную и конечную дату
start_date = input("\nВведите начальную дату (дд-мм-гггг): ")
end_date = input("Введите конечную дату (дд-мм-гггг): ")

# 7. Преобразуем введенные даты в формат datetime
start_date = pd.to_datetime(start_date, format='%d-%m-%Y')
end_date = pd.to_datetime(end_date, format='%d-%m-%Y')

# 8. Фильтруем данные по указанному диапазону
filtered_df = df.loc[start_date:end_date]

# 9. Выводим результат
print("\nВременной ряд за указанный период:")
print(filtered_df)

Сгенерированный временной ряд:
            Значение
Дата                
2023-01-01       103
2023-01-02       114
2023-01-03       165
2023-01-04       117
2023-01-05       131
...              ...
2023-12-27       149
2023-12-28       110
2023-12-29       162
2023-12-30       108
2023-12-31       106

[365 rows x 1 columns]

Введите начальную дату (дд-мм-гггг): 01-01-2023
Введите конечную дату (дд-мм-гггг): 05-01-2023

Временной ряд за указанный период:
            Значение
Дата                
2023-01-01       103
2023-01-02       114
2023-01-03       165
2023-01-04       117
2023-01-05       131


**2. Работа с несколькими DataFrame**

Напишите программу, которая:
1. Создаёт DataFrame с тремя столбцами: "Товар", “Откуда”, “Куда” (в последних двух столбцах указываются города).
1. Создаёт второй DataFrame с двумя столбцами: "Город", “Страна”.
1. На основе слияния двух предыдущих DataFrame создает третий, в котором для каждого товара вместо городов доставки будут указаны страны. Лишних столбцов в этом DataFrame быть не должно, только "Товар", “Откуда”, “Куда” (в последних двух столбцах указываются страны)
1. Добавьте в последний DataFrame новый столбец “Внутри_страны”, который принимает логические значения (True, если доставка осуществляется внутри страны).
1. Вычислите, сколько в наших данных доставок внутри страны.

In [ ]:
import pandas as pd

# 1. Создаем первый DataFrame с товарами и городами
df_routes = pd.DataFrame({
    "Товар": ["Ноутбук", "Смартфон", "Кофеварка", "Наушники", "Принтер"],
    "Откуда": ["Москва", "Минск", "Берлин", "Санкт-Петербург", "Мюнхен"],
    "Куда":   ["Санкт-Петербург", "Брест", "Минск", "Москва", "Берлин"]
})

# 2. Создаем второй DataFrame с соответствием городов и стран
df_cities = pd.DataFrame({
    "Город":   ["Москва", "Санкт-Петербург", "Минск", "Брест", "Берлин", "Мюнхен"],
    "Страна":  ["Россия", "Россия", "Беларусь", "Беларусь", "Германия", "Германия"]
})

# 3. Соединяем маршруты с таблицей городов, чтобы получить страну отправки
left_cities = df_cities.add_prefix("Откуда_")   # колонки станут Откуда_Город, Откуда_Страна
tmp = df_routes.merge(left_cities, left_on="Откуда", right_on="Откуда_Город", how="left")

# 4. Соединяем результат с таблицей городов, чтобы получить страну назначения
right_cities = df_cities.add_prefix("Куда_")    # колонки станут Куда_Город, Куда_Страна
tmp = tmp.merge(right_cities, left_on="Куда", right_on="Куда_Город", how="left")

# 5. Оставляем только нужные столбцы: "Товар", "Откуда", "Куда" (теперь это страны)
df_final = tmp[["Товар", "Откуда_Страна", "Куда_Страна"]].rename(
    columns={"Откуда_Страна": "Откуда", "Куда_Страна": "Куда"}
)

# 6. Добавляем столбец "Внутри_страны" (True, если страна отправки = страна назначения)
df_final["Внутри_страны"] = df_final["Откуда"] == df_final["Куда"]

# 7. Считаем количество доставок внутри страны
inside_count = df_final["Внутри_страны"].sum()

# 8. Выводим результаты
print("Изначальные данные (товар и города):")
print(df_routes)

print("\nСправочник городов:")
print(df_cities)

print("\nИтоговый DataFrame (страны вместо городов):")
print(df_final)

print("\nКоличество доставок внутри страны:", inside_count)

Изначальные данные (товар и города):
       Товар           Откуда             Куда
0    Ноутбук           Москва  Санкт-Петербург
1   Смартфон            Минск            Брест
2  Кофеварка           Берлин            Минск
3   Наушники  Санкт-Петербург           Москва
4    Принтер           Мюнхен           Берлин

Справочник городов:
             Город    Страна
0           Москва    Россия
1  Санкт-Петербург    Россия
2            Минск  Беларусь
3            Брест  Беларусь
4           Берлин  Германия
5           Мюнхен  Германия

Итоговый DataFrame (страны вместо городов):
       Товар    Откуда      Куда  Внутри_страны
0    Ноутбук    Россия    Россия           True
1   Смартфон  Беларусь  Беларусь           True
2  Кофеварка  Германия  Беларусь          False
3   Наушники    Россия    Россия           True
4    Принтер  Германия  Германия           True

Количество доставок внутри страны: 4


**3. Напишите программу, которая:**
1. Запрашивает у пользователя количество строк данных для ввода.
1. Для каждой строки запрашивает данные по трем столбцам: "Имя", "Возраст", "Регион", разделённые пробелом.
1. Создает DataFrame на основе введенных данных.
1. Выводит общую информацию о DataFrame, включая количество записей, средний возраст, и количество уникальных регионов.
1. Выбирает и выводит на экран данные о самом молодом участнике.

In [ ]:
import pandas as pd

# 1. Запрашиваем количество строк
n = int(input("Введите количество строк данных: "))

# 2. Вводим данные построчно
data = []
for i in range(n):
    row = input(f"Введите данные {i+1}-й строки (Имя Возраст Регион): ").split()
    if len(row) != 3:
        print("Нужно ввести три значения через пробел. Попробуйте снова.")
        row = input(f"Введите данные {i+1}-й строки (Имя Возраст Регион): ").split()
    data.append(row)

# 3. Создаем DataFrame
df = pd.DataFrame(data, columns=["Имя", "Возраст", "Регион"])

# 4. Преобразуем столбец "Возраст" в числовой формат
df["Возраст"] = df["Возраст"].astype(int)

# 5. Выводим общую информацию
print("\nОбщая информация о данных:")
print("Количество записей:", len(df))
print("Средний возраст:", df["Возраст"].mean())
print("Количество уникальных регионов:", df["Регион"].nunique())

# 6. Находим самого молодого участника
youngest = df.loc[df["Возраст"].idxmin()]

print("\nСамый молодой участник:")
print(youngest)

Введите количество строк данных: 3
Введите данные 1-й строки (Имя Возраст Регион): Анна 25 Москва
Введите данные 2-й строки (Имя Возраст Регион): Борис 30 Минск
Введите данные 3-й строки (Имя Возраст Регион): Ольга 22 Киев

Общая информация о данных:
Количество записей: 3
Средний возраст: 25.666666666666668
Количество уникальных регионов: 3

Самый молодой участник:
Имя        Ольга
Возраст       22
Регион      Киев
Name: 2, dtype: object
